#### Import Libraries

In [1]:
import numpy as np 
import xarray as xr
from os.path import join 

#### Path variables

In [2]:
import os
os.getcwd()

'/home/kevin/Escritorio/Kevin/workspace/CRAI_hourly_infilling/WindData_Preprocessing/5.-Data_Split_train_val_test'

In [3]:
path = '/home/kevin/Escritorio/Kevin/workspace/CRAI_hourly_infilling/WindData_Preprocessing/'
input_path_era = "4.-Split_Data_Can_Pen/"
input_path_mask = "4.-Split_Data_Can_Pen/"
data_file = "era5SL_010_144x144_m_s_2011_2021_pen.nc"
mask_file = "aemet_mask_daily-wind_m_s_2011-2021_010x010_144_pen.nc"

exp_name_data = "ws" # wind speed
exp_name_mask = "mask" # wind speed
code_name_data = "ws"
code_name_mask = "ws"

#### Clean ERA5-Land data: convert NaNs to 0's and remove coordinate expver (indicating final or not final ERA5-Land data)

In [4]:
dataset = xr.open_dataset(join(path, input_path_era, data_file))

In [5]:
dataset['ws'].values

array([[[       nan,        nan,        nan, ..., 0.6822175 ,
         0.84097415, 1.0057882 ],
        [       nan,        nan,        nan, ..., 0.7513809 ,
         0.79647547, 0.8910398 ],
        [       nan,        nan,        nan, ..., 0.94734585,
         0.93201363, 0.9503836 ],
        ...,
        [       nan,        nan,        nan, ..., 3.065885  ,
         2.8773074 , 2.6976206 ],
        [       nan,        nan, 3.0000088 , ..., 3.1961434 ,
         3.0398521 , 2.8838844 ],
        [       nan,        nan, 2.8552024 , ..., 3.3149717 ,
         3.184975  , 3.0494797 ]],

       [[       nan,        nan,        nan, ..., 0.6343377 ,
         0.72571427, 0.8475767 ],
        [       nan,        nan,        nan, ..., 0.7046255 ,
         0.69525725, 0.74653184],
        [       nan,        nan,        nan, ..., 0.8895084 ,
         0.8450012 , 0.83395594],
        ...,
        [       nan,        nan,        nan, ..., 3.304483  ,
         3.2501872 , 3.1840441 ],
        [   

In [6]:
dataset_0 = dataset.where(dataset['ws'].isnull()==False, 0)

In [7]:
dataset_0['ws'].values

array([[[0.        , 0.        , 0.        , ..., 0.6822175 ,
         0.84097415, 1.0057882 ],
        [0.        , 0.        , 0.        , ..., 0.7513809 ,
         0.79647547, 0.8910398 ],
        [0.        , 0.        , 0.        , ..., 0.94734585,
         0.93201363, 0.9503836 ],
        ...,
        [0.        , 0.        , 0.        , ..., 3.065885  ,
         2.8773074 , 2.6976206 ],
        [0.        , 0.        , 3.0000088 , ..., 3.1961434 ,
         3.0398521 , 2.8838844 ],
        [0.        , 0.        , 2.8552024 , ..., 3.3149717 ,
         3.184975  , 3.0494797 ]],

       [[0.        , 0.        , 0.        , ..., 0.6343377 ,
         0.72571427, 0.8475767 ],
        [0.        , 0.        , 0.        , ..., 0.7046255 ,
         0.69525725, 0.74653184],
        [0.        , 0.        , 0.        , ..., 0.8895084 ,
         0.8450012 , 0.83395594],
        ...,
        [0.        , 0.        , 0.        , ..., 3.304483  ,
         3.2501872 , 3.1840441 ],
        [0. 

In [8]:
dataset_cleaned = dataset_0.drop_vars('expver')

In [9]:
dataset_cleaned

<xarray.Dataset> Size: 8GB
Dimensions:    (time: 96432, latitude: 144, longitude: 144)
Coordinates:
    number     int64 8B 0
  * time       (time) datetime64[ns] 771kB 2011-01-01 ... 2021-12-31T23:00:00
  * latitude   (latitude) float64 1kB 45.9 45.8 45.7 45.6 ... 31.8 31.7 31.6
  * longitude  (longitude) float64 1kB -9.8 -9.7 -9.6 -9.5 ... 4.2 4.3 4.4 4.5
Data variables:
    ws         (time, latitude, longitude) float32 8GB 0.0 0.0 ... 3.547 3.523
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-02-06T15:24 GRIB to CDM+CF via cfgrib-0.9.1...

In [10]:
dataset_cleaned.attrs

{'GRIB_centre': 'ecmf',
 'GRIB_centreDescription': 'European Centre for Medium-Range Weather Forecasts',
 'GRIB_subCentre': np.int64(0),
 'Conventions': 'CF-1.7',
 'institution': 'European Centre for Medium-Range Weather Forecasts',
 'history': '2025-02-06T15:24 GRIB to CDM+CF via cfgrib-0.9.15.0/ecCodes-2.39.0 with {"source": "data.grib", "filter_by_keys": {}, "encode_cf": ["parameter", "time", "geography", "vertical"]}'}

#### Split data randomly

In [11]:
def split_data_randomly(mask_path, dataset_cleaned, opath, split_ratio=0.7, split_test_1=0.5): #, split_test_2=0.5):

    dataset = dataset_cleaned #xr.open_dataset(data_path)
    
    mask_dataset = xr.open_dataset(mask_path)
    #dataset = dataset.rename({exp_name_data: code_name_data})
    #mask_dataset = mask_dataset.rename({exp_name_mask: code_name_mask})

    hdata = dataset['ws']
    hmask = mask_dataset['ws']

    print("Shape of the data: ", hdata.shape)
    print("Shape of the mask: ", hmask.shape)

    shuffle_index = np.random.permutation(len(hdata))
    #print(shuffle_index[:10])

    train_index = np.sort(shuffle_index[:int(len(hdata) * split_ratio)])
    rest_index = np.sort(shuffle_index[int(len(hdata) * split_ratio):])
    val_index = np.sort(rest_index[:int(len(rest_index) * split_test_1)])
    rest_index = np.sort(rest_index[int(len(rest_index) * split_test_1):])
    test_index = np.sort(rest_index[:int(len(rest_index))])
    # test_2_index = np.sort(rest_index[int(len(rest_index) * split_test_2):])

    print()
    print("Len of train dataset: ", len(train_index))
    print("Len of val dataset: ", len(val_index))
    print("Len of test 1 dataset: ", len(test_index))
    # print("Len of test 2 dataset: ", len(test_2_index))

    train_data = hdata[train_index]
    val_data = hdata[val_index]
    test_data = hdata[test_index]
    # test_2_data = hdata[test_2_index]

    train_mask = hmask[train_index]
    val_mask = hmask[val_index]
    test_mask = hmask[test_index]
    # test_2_mask = hmask[test_2_index]

    print()
    print(opath)
    print()

    train_data.to_netcdf(opath + "train_data_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_"  + ".nc")
    val_data.to_netcdf(opath + "val_data_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_"  + ".nc")
    test_data.to_netcdf(opath + "test1_data_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_" + ".nc")
    # test_2_data.to_netcdf(opath + "test2_data_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_" + str(split_test_2) + ".nc")

    train_mask.to_netcdf(opath + "train_mask_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_"  + ".nc")
    val_mask.to_netcdf(opath + "val_mask_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_"  + ".nc")
    test_mask.to_netcdf(opath + "test_mask_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_" + ".nc")   
    # test_2_mask.to_netcdf(opath + "test_mask_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_" + str(split_test_2) + ".nc")   

    return hmask, hdata
       

#### Split data by years

In [17]:
def split_data_byyears(mask_path, dataset_cleaned, opath, years):

    hdata = dataset_cleaned #xr.open_dataset(data_path)
    hmask = xr.open_dataset(mask_path)
    #dataset = dataset.rename({exp_name_data: code_name_data}) #needed in the kadow code
    #mask_dataset = mask_dataset.rename({exp_name_mask: code_name_mask}) #needed in the kadow code

    # hdata = dataset['ws']
    # hmask = mask_dataset['ws']

    print("Shape of the data: ", hdata['ws'].shape)
    print("Shape of the mask: ", hmask['ws'].shape)

    train_data = hdata.sel(time=slice(str(years[0]), str(years[1])))
    val_data = hdata.sel(time=slice(str(years[1]+1), str(years[2])))
    test_data = hdata.sel(time=slice(str(years[2]+ 1), str(years[3])))
    # test_data_2 = hdata.sel(time=slice(str(years[3]+ 1), str(years[4])))

    train_mask = hmask.sel(time=slice(str(years[0]), str(years[1])))
    val_mask = hmask.sel(time=slice(str(years[1] + 1), str(years[2])))
    test_mask = hmask.sel(time=slice(str(years[2] + 1), str(years[3])))
    # test_mask_2 = hmask.sel(time=slice(str(years[3] + 1), str(years[4])))

    print()
    print("Shape of the train data: ", train_data['ws'].shape)
    print("Shape of the train mask: ", train_mask['ws'].shape)
    print("Shape of the val data: ", val_data['ws'].shape)
    print("Shape of the val mask: ", val_mask['ws'].shape)
    print("Shape of the test data: ", test_data['ws'].shape)
    print("Shape of the test mask: ", test_mask['ws'].shape)
    # print("Shape of the test_2 data: ", test_data_2.shape)
    #print("Shape of the test_2 mask: ", test_mask_2.shape)

    train_data.to_netcdf(opath + "train_data_m_s_" + str(years[0]) + "_" + str(years[1]) + "_pen" + ".nc")
    val_data.to_netcdf(opath + "val_data_m_s_" + str(years[1] + 1) + "_" + str(years[2]) + "_pen" + ".nc")
    test_data.to_netcdf(opath + "test_data_m_s_" + str(years[2] + 1) + "_" + str(years[3]) + "_pen" + ".nc")
    # test_data_2.to_netcdf(opath + "test_2_data_m_s_" + str(years[3] + 1) + "_" + str(years[4]) + ".nc")

    train_mask.to_netcdf(opath + "train_mask_m_s_" + str(years[0]) + "_" + str(years[1]) + "_pen" + ".nc")
    val_mask.to_netcdf(opath + "val_mask_m_s_" + str(years[1] + 1) + "_" + str(years[2]) + "_pen" + ".nc")
    test_mask.to_netcdf(opath + "test_mask_m_s_" + str(years[2] + 1) + "_" + str(years[3]) + "_pen" + ".nc")
    # test_mask_2.to_netcdf(opath + "test_2_mask_m_s_" + str(years[3] + 1) + "_" + str(years[4]) + ".nc")

    print(train_data)
    print(train_mask)

    print(test_data)
    print(test_mask)

    return hmask, hdata  


#### Main

In [18]:
if __name__ == '__main__':
    np.random.seed(42)

    split_ratio = 0.7
    split_test_1 = 0.5
    split_test_2 = 0.5

    years = [2011, 2019, 2020, 2021]

    output_path_year = "5.-Data_Split_train_val_test/Split_data_meters_second_" + str(years[0]) + "-" + str(years[-1]) + "/ByYear/"
    output_path_random =  "5.-Data_Split_train_val_test/Split_data_meters_second_" + str(years[0]) + "-" + str(years[-1]) + "/Randomly/"

    if not os.path.exists(path + output_path_year):
        os.makedirs(path + output_path_year)

    if not os.path.exists(path + output_path_random):
        os.makedirs(path + output_path_random)

    mask, data = split_data_byyears(path + input_path_mask + mask_file, dataset_cleaned, path + output_path_year, years)

    print()

    mask, data = split_data_randomly(path + input_path_mask + mask_file, dataset_cleaned, path + output_path_random, split_ratio, split_test_1) #, split_test_2)


Shape of the data:  (96432, 144, 144)
Shape of the mask:  (4018, 144, 144)

Shape of the train data:  (78888, 144, 144)
Shape of the train mask:  (3287, 144, 144)
Shape of the val data:  (8784, 144, 144)
Shape of the val mask:  (366, 144, 144)
Shape of the test data:  (8760, 144, 144)
Shape of the test mask:  (365, 144, 144)
<xarray.Dataset> Size: 7GB
Dimensions:    (time: 78888, latitude: 144, longitude: 144)
Coordinates:
    number     int64 8B 0
  * time       (time) datetime64[ns] 631kB 2011-01-01 ... 2019-12-31T23:00:00
  * latitude   (latitude) float64 1kB 45.9 45.8 45.7 45.6 ... 31.8 31.7 31.6
  * longitude  (longitude) float64 1kB -9.8 -9.7 -9.6 -9.5 ... 4.2 4.3 4.4 4.5
Data variables:
    ws         (time, latitude, longitude) float32 7GB 0.0 0.0 ... 1.43 1.381
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:          

IndexError: index 4020 is out of bounds for axis 0 with size 4018

In [ ]:
3287*24

78888

In [ ]:
366*24

8784

In [ ]:
365*24

8760

In [ ]:
4018*24

96432